# Query Model

Use the model to predict the outcome of a new patient.
Patients from the test set are drawn at random.

Before running this notebook, you need to either start the flask server:

```bash
python scripts/predict.py
```

or build the docker image and run the container:

```bash
docker build -t diabetes_readmission .
docker run -p 9696:9696 diabetes_readmission
```

In [1]:
# dependencies
import requests
import pandas as pd
import numpy as np
from pathlib import Path
# API endpoint
URL = "http://localhost:9696/predict"

# paths
PATH_DATA = Path("../data/processed")
PATH_TEST = PATH_DATA / "test"

# load test data
X_test = pd.read_csv(PATH_TEST / "X_test.csv")
y_test = pd.read_csv(PATH_TEST / "y_test.csv")

# get a test case including label
patient_index = np.random.randint(0, len(X_test))
test_case = {
    "features": X_test.iloc[patient_index].to_dict(),
    "label": y_test.iloc[patient_index]
}

# display the test case
print(f"Features: {test_case['features']}")
print(f"Label: {test_case['label']}")

Features: {'time_in_hospital': 0.4162896638551851, 'num_lab_procedures': 0.4499925682277403, 'num_procedures': 0.0, 'num_medications': 0.6005433756755866, 'number_outpatient': 0.0, 'number_emergency': 0.0, 'number_inpatient': 0.0, 'number_diagnoses': 0.7924812503553146, 'change': 1.0, 'diabetesmed': 1.0, 'race_africanamerican': 0.0, 'race_asian': 0.0, 'race_caucasian': 0.0, 'race_hispanic': 0.0, 'race_other': 1.0, 'gender_female': 0.0, 'gender_male': 1.0, 'age_70+': 1.0, 'age_[0_10)': 0.0, 'age_[10_20)': 0.0, 'age_[20_50)': 0.0, 'age_[50_70)': 0.0, 'admission_type_id_elective': 0.0, 'admission_type_id_emergency': 1.0, 'admission_type_id_new born': 0.0, 'admission_type_id_trauma center': 0.0, 'discharge_disposition_id_discharged to home': 1.0, 'discharge_disposition_id_other': 0.0, 'admission_source_id_emergency': 0.0, 'admission_source_id_other': 0.0, 'admission_source_id_referral': 1.0, 'admission_source_id_transfer': 0.0, 'medical_specialty_cardiology': 0.0, 'medical_specialty_emerge

In [2]:
def make_prediction(test_case):
    """
    Make a prediction for a test case and print the results.
    """
    
    # make prediction
    response = requests.post(URL, json=test_case["features"])
    
    # get true label
    true_label = (
        "readmitted" if test_case["label"].values[0] == 1 
        else "not_readmitted"
    )
    
    # check if prediction is correct
    if response.json()['predicted_outcome'] == true_label:
        message = "Nice, the prediction is correct!"
    else:
        message = "Gah, the prediction is incorrect! We're sorry!"
    
    # print prediction
    print(
        f"Predicted outcome: "
        f"{response.json()['predicted_outcome'].replace('_', ' ')} " 
        f"(Probability: "
        f"{response.json()['predicted_probability'][0][1]:.4f})\n"
        f"Real outcome: {true_label.replace('_', ' ')}\n"
        f"{message}"
    )

In [3]:
# make prediction
make_prediction(test_case)

Predicted outcome: not readmitted (Probability: 0.4289)
Real outcome: not readmitted
Nice, the prediction is correct!
